In [15]:
import torch
from huggingface_hub import login, whoami
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
path = "/nlsasfs/home/isea/isea10/aansh/deception_detection/weights/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(path)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
import json

with open("/nlsasfs/home/isea/isea10/aansh/introspection/data/exp1/single_token_completion_prompts.json") as f:
    prompts = json.load(f)
    
message = []
prompt_x = prompts[0]

In [4]:
prompt_x

{'prompt': 'Name a fruit.', 'completion_1': 'Apple', 'completion_2': 'Banana'}

In [5]:
ls_1 = {"message": [], "metadata": {}}
ls_1["message"].append({"role": "user", "content": prompt_x["prompt"]})
ls_1["message"].append({"role": "assistant", "content": prompt_x["completion_1"]})

ls_2 = {"message": [], "metadata": {}}
ls_2["message"].append({"role": "user", "content": prompt_x["prompt"]})
ls_2["message"].append({"role": "assistant", "content": prompt_x["completion_2"]})

In [8]:
inputs = tokenizer.apply_chat_template(ls_1["message"], tokenize=False)
input_ids = tokenizer(inputs, return_tensors="pt")
input_ids

{'input_ids': tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   1627,  10263,    220,   2366,     19,    271, 128009,
         128006,    882, 128007,    271,    678,    264,  14098,     13, 128009,
         128006,  78191, 128007,    271,  27665, 128009]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [20]:
tokenizer.decode(input_ids["input_ids"][0][-3])

'\n\n'

In [11]:
outputs = model(**input_ids)
logits = outputs.logits
logits

tensor([[[-3.1094,  1.5234,  7.3438,  ...,  1.7500,  1.7500,  1.7500],
         [-3.1094,  1.5234,  7.3438,  ...,  1.7500,  1.7500,  1.7500],
         [-1.7031, -0.6914, -4.1875,  ...,  3.4688,  3.4688,  3.4688],
         ...,
         [ 5.0312, 16.5000,  5.2500,  ..., -0.0762, -0.0762, -0.0767],
         [12.0625,  7.6562,  0.9023,  ...,  3.8125,  3.8125,  3.8125],
         [-7.0625, -5.0938, -8.8125,  ...,  9.2500,  9.2500,  9.2500]]],
       dtype=torch.bfloat16, grad_fn=<ToCopyBackward0>)

In [27]:
fruit_place_logits = logits[0, -3, :]
top_10_indices = torch.argsort(fruit_place_logits, descending=True)[:10]
for index in top_10_indices:
    token = tokenizer.decode(index)
    score = fruit_place_logits[index].item()
    print(f"Token: {token}, Score: {score}")

Token: P, Score: 21.625
Token: Apple, Score: 21.375
Token: Orange, Score: 20.875
Token: M, Score: 20.125
Token: L, Score: 20.125
Token: Str, Score: 20.125
Token: Ban, Score: 19.125
Token: A, Score: 19.125
Token: G, Score: 19.0
Token: Pear, Score: 18.875


In [30]:
tokenizer.decode(tokenizer.encode("Banana")[2])

'ana'